# Anomaly Detection Analysis

This notebook identifies unusual patterns and operational signals in Aadhaar data.

## Methods Used:
1. **Z-Score Analysis**: Statistical outlier detection
2. **IQR Method**: Interquartile range-based anomalies
3. **Time-Series Anomalies**: Unexpected changes in trends
4. **Multi-dimensional Analysis**: Combined metric anomalies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.ensemble import IsolationForest

print("✓ Libraries imported")

## 1. Load Data

In [ ]:
df = pd.read_csv('../data/processed/data_with_indicators.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Loaded {len(df):,} records with indicators")
df.head()

## 2. Z-Score Based Anomaly Detection

In [ ]:
# Calculate Z-scores for key metrics
metrics = ['total_enrolment', 'total_updates', 'update_pressure_index']

for metric in metrics:
    df[f'{metric}_zscore'] = np.abs(stats.zscore(df[metric].fillna(0)))

# Flag anomalies (Z-score > 3)
threshold = 3
df['is_anomaly'] = (df[[f'{m}_zscore' for m in metrics]].max(axis=1) > threshold)

print(f"\n🔍 Anomalies Detected: {df['is_anomaly'].sum():,} ({df['is_anomaly'].mean()*100:.2f}%)")

# Show anomaly distribution by metric
for metric in metrics:
    anomaly_count = (df[f'{metric}_zscore'] > threshold).sum()
    print(f"  - {metric}: {anomaly_count} anomalies")

## 3. Temporal Anomaly Patterns

In [ ]:
# Anomalies over time
daily_anomalies = df.groupby('date')['is_anomaly'].sum().reset_index()
daily_anomalies.columns = ['date', 'anomaly_count']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_anomalies['date'],
    y=daily_anomalies['anomaly_count'],
    mode='lines+markers',
    name='Daily Anomaly Count',
    line=dict(color='red', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='Operational Signals (Anomalies) Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Anomalies',
    height=500
)

fig.show()

print("\n📊 Temporal Pattern:")
print(f"Peak anomaly date: {daily_anomalies.loc[daily_anomalies['anomaly_count'].idxmax(), 'date']}")
print(f"Average anomalies per day: {daily_anomalies['anomaly_count'].mean():.1f}")

## 4. Geographic Distribution of Anomalies

In [ ]:
# State-wise anomaly count
state_anomalies = df[df['is_anomaly']].groupby('state').size().reset_index(name='anomaly_count')
state_anomalies = state_anomalies.sort_values('anomaly_count', ascending=False)

fig = px.bar(
    state_anomalies.head(15),
    x='state',
    y='anomaly_count',
    title='Top 15 States by Operational Signals',
    labels={'anomaly_count': 'Number of Signals', 'state': 'State'},
    color='anomaly_count',
    color_continuous_scale='Oranges'
)

fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

## 5. Detailed Anomaly Analysis

In [ ]:
# Top anomalies by different metrics
print("\n🔴 Top Anomalies by Enrollment:\n")
top_enrollment_anomalies = df.nlargest(10, 'total_enrolment_zscore')[[
    'date', 'state', 'district', 'total_enrolment', 'total_enrolment_zscore'
]]
print(top_enrollment_anomalies)

print("\n🔴 Top Anomalies by Update Pressure Index:\n")
top_upi_anomalies = df.nlargest(10, 'update_pressure_index_zscore')[[
    'date', 'state', 'district', 'update_pressure_index', 'update_pressure_index_zscore'
]]
print(top_upi_anomalies)

## 6. Repeated vs Single Anomalies

In [ ]:
# Districts with repeated anomalies
district_anomaly_freq = df[df['is_anomaly']].groupby(['state', 'district']).size().reset_index(name='frequency')
district_anomaly_freq = district_anomaly_freq.sort_values('frequency', ascending=False)

print("\n🔴 Districts with Repeated Anomalies (Priority: HIGH):\n")
repeated_anomalies = district_anomaly_freq[district_anomaly_freq['frequency'] > 1].head(15)
print(repeated_anomalies)

print(f"\n📊 Summary:")
print(f"Districts with single anomalies: {(district_anomaly_freq['frequency'] == 1).sum()}")
print(f"Districts with repeated anomalies: {(district_anomaly_freq['frequency'] > 1).sum()}")
print(f"Max anomaly frequency: {district_anomaly_freq['frequency'].max()} occurrences")

## 7. Isolation Forest (Advanced)

In [ ]:
# Use Isolation Forest for multi-dimensional anomaly detection
feature_cols = ['total_enrolment', 'total_updates', 'update_pressure_index']
X = df[feature_cols].fillna(0)

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
df['iso_forest_anomaly'] = iso_forest.fit_predict(X)
df['iso_forest_anomaly'] = df['iso_forest_anomaly'] == -1

print(f"\n🤖 Isolation Forest Anomalies: {df['iso_forest_anomaly'].sum():,}")

# Compare methods
print("\n📊 Method Comparison:")
print(f"Z-Score method: {df['is_anomaly'].sum():,} anomalies")
print(f"Isolation Forest: {df['iso_forest_anomaly'].sum():,} anomalies")
print(f"Overlap: {(df['is_anomaly'] & df['iso_forest_anomaly']).sum():,} common anomalies")

## 8. Anomaly Visualization

In [ ]:
# 2D visualization of anomalies
fig = px.scatter(
    df.sample(min(5000, len(df))),  # Sample for performance
    x='total_enrolment',
    y='total_updates',
    color='is_anomaly',
    title='Anomaly Detection: Enrollments vs Updates',
    labels={'total_enrolment': 'Total Enrollments', 'total_updates': 'Total Updates'},
    color_discrete_map={True: 'red', False: 'blue'},
    hover_data=['state', 'district', 'update_pressure_index']
)

fig.update_layout(height=600)
fig.show()

## 9. Operational Insights

### Key Findings:

1. **Anomaly Frequency**: X% of records flagged as operational signals
2. **Geographic Concentration**: Certain states show higher anomaly rates
3. **Temporal Patterns**: Anomalies cluster around specific time periods
4. **Repeated Signals**: Some districts consistently show unusual patterns

### Recommended Actions:

**Priority: 🔴 HIGH** (Repeated Anomalies)
- Investigate districts with multiple anomaly occurrences
- Check for data quality issues or operational irregularities
- Deploy additional resources if genuine demand spikes

**Priority: 🟡 MEDIUM** (Single Anomalies)
- Monitor for recurrence
- Review operational logs for the anomaly dates
- Validate data capture processes

**Priority: 🟢 LOW** (Machine Learning Detected)
- Review Isolation Forest anomalies for subtle patterns
- Use for predictive maintenance and planning

### Data Quality Note:
Anomalies may indicate:
- Genuine operational load spikes
- Data entry errors
- Special events (campaigns, holidays)
- System issues

**Next Step**: Manual validation required for high-priority anomalies

In [ ]:
# Save anomaly-flagged data
df.to_csv('../data/processed/data_with_anomalies.csv', index=False)
print("\n✅ Anomaly detection complete!")
print("Data saved to: data/processed/data_with_anomalies.csv")
print("\nNext: Proceed to 05_forecasting.ipynb for predictive modeling")